In [1]:
import numpy as np
import tensorflow as tf
from tf_agents.networks.q_network import QNetwork
import tf_agents.networks.network as network
from tf_agents.specs import tensor_spec
from snake_game import SnakeGame
from scene import Scene
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input

scene = Scene(init_randomly=True)
episodes_count = 50000
random_episodes = 20000


In [2]:
from tf_agents.environments.tf_py_environment import TFPyEnvironment

env = SnakeGame(scene)
env = TFPyEnvironment(env)


In [3]:
class MyQNetwork(network.Network):
    def __init__(self,
                 input_tensor_spec,
                 action_spec,
                 name='MyQNetwork'):
        super(MyQNetwork, self).__init__(
            input_tensor_spec=input_tensor_spec,
            state_spec=(),
            name=name)
        
        self._action_spec = action_spec
        matrix_shape = input_tensor_spec[0].shape
        array_shape = input_tensor_spec[1].shape
        
        input1 = Input(shape=matrix_shape)
        input2 = Input(shape=array_shape) # Each value represents if there is a wall or the snake body in the direction, 0 - no wall, 1 - wall
        
        conv = Conv2D(16, (2, 2), 1, activation='relu', kernel_initializer='he_normal')(input1)
        flat = Flatten()(conv)
        dense1 = Dense(64, activation='relu', kernel_initializer='he_normal')(flat)
        dense2 = Dense(16, activation='relu', kernel_initializer='he_normal')(dense1)
        
        concat = tf.keras.layers.concatenate([dense2, input2])
        output = Dense(4, activation='linear')(concat)
        
        self._model = tf.keras.Model(inputs=[input1, input2], outputs=output)
        
        self._model.summary()

    def call(self, observations, step_type=None, network_state=(), training=False):
        output = self._model([observations[0], observations[1]])
        return output, network_state

q_net = MyQNetwork(
    env.observation_spec(),
    env.action_spec())


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 5, 5, 4)]            0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 4, 4, 16)             272       ['input_1[0][0]']             
                                                                                                  
 flatten (Flatten)           (None, 256)                  0         ['conv2d[0][0]']              
                                                                                                  
 dense (Dense)               (None, 64)                   16448     ['flatten[0][0]']             
                                                                                              

In [4]:
# Create the agent
from tf_agents.agents.dqn.dqn_agent import DdqnAgent
from tensorflow import keras
from keras.optimizers.legacy import Adam
from keras.losses import Huber
from tf_agents.trajectories import TimeStep

train_step_counter = tf.Variable(0)
optimizer = Adam(learning_rate=0.003)

epsilon = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=0.1,
    decay_steps=episodes_count,
    end_learning_rate=0.01
)

agent = DdqnAgent(
    time_step_spec=env.time_step_spec(),
    action_spec=env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=Huber(reduction="none"),
    gamma=tf.constant(0.95, dtype=tf.float32),
    train_step_counter=train_step_counter,
    epsilon_greedy=lambda: epsilon(train_step_counter)
)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 5, 5, 4)]            0         []                            
                                                                                                  
 conv2d_1 (Conv2D)           (None, 4, 4, 16)             272       ['input_3[0][0]']             
                                                                                                  
 flatten_1 (Flatten)         (None, 256)                  0         ['conv2d_1[0][0]']            
                                                                                                  
 dense_3 (Dense)             (None, 64)                   16448     ['flatten_1[0][0]']           
                                                                                            

In [5]:
# Create the replay buffer
from tf_agents.replay_buffers import tf_uniform_replay_buffer

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=env.batch_size,
    max_length=100000
)


In [6]:
# Create the training metrics
from tf_agents.metrics import tf_metrics

train_metrics = [
    tf_metrics.NumberOfEpisodes(),
    tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(),
    tf_metrics.AverageEpisodeLengthMetric()
]


In [7]:
# Create the driver
from tf_agents.drivers import dynamic_step_driver

collect_driver = dynamic_step_driver.DynamicStepDriver(
    env,
    policy=agent.collect_policy,
    observers=[replay_buffer.add_batch] + train_metrics,
    num_steps=4
)

# Run a random policy to fill the replay buffer
from tf_agents.policies.random_tf_policy import RandomTFPolicy

initial_collect_policy = RandomTFPolicy(env.time_step_spec(), env.action_spec())
init_driver = dynamic_step_driver.DynamicStepDriver(
    env,
    policy=initial_collect_policy,
    observers=[replay_buffer.add_batch],
    num_steps=random_episodes
)

final_time_step, final_policy_state = init_driver.run()


In [ ]:
# Create the dataset
dataset = replay_buffer.as_dataset(
    sample_batch_size=64,
    num_steps=2, # Capaz cambiar esto
    num_parallel_calls=3
).prefetch(3)


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [ ]:
# Create the training loop

from tf_agents.utils.common import function
from tf_agents.trajectories import trajectory
from tf_agents.eval.metric_utils import log_metrics
import logging

logging.getLogger().setLevel(logging.INFO)

collect_driver.run = function(collect_driver.run)
agent.train = function(agent.train)

def train_agent(n_iterations):
    time_step = None
    policy_state = agent.collect_policy.get_initial_state(env.batch_size)
    iterator = iter(dataset)
    for iteration in range(n_iterations):
        time_step, policy_state = collect_driver.run(time_step, policy_state)
        trajectories, buffer_info = next(iterator)
        train_loss = agent.train(trajectories)
        if iteration % 1000 == 0:
            print("Iteration: ", iteration)
            log_metrics(train_metrics)


In [ ]:
train_agent(episodes_count)


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


INFO:absl: 
		 NumberOfEpisodes = 4
		 EnvironmentSteps = 4
		 AverageReturn = -50.0
		 AverageEpisodeLength = 0.75


Iteration:  0


INFO:absl: 
		 NumberOfEpisodes = 343
		 EnvironmentSteps = 4004
		 AverageReturn = -6028.0
		 AverageEpisodeLength = 30.0


Iteration:  1000


...

Iteration:  2000


INFO:absl: 
		 NumberOfEpisodes = 580
		 EnvironmentSteps = 12004
		 AverageReturn = -8370.0
		 AverageEpisodeLength = 51.29999923706055


Iteration:  3000


INFO:absl: 
		 NumberOfEpisodes = 643
		 EnvironmentSteps = 16004
		 AverageReturn = -8354.5
		 AverageEpisodeLength = 70.19999694824219


Iteration:  4000


In [ ]:
# Evaluate the agent

time_step = env.reset()
rewards = []
steps = 0

while not time_step.is_last() and steps < 200:
    steps += 1
    print("Map:\n", np.argmax(time_step.observation[0], axis=3))
    print("Direction:\n", time_step.observation[1])
    action_step = agent.policy.action(time_step)
    print("Action: ", action_step.action.numpy())
    time_step = env.step(action_step.action)
    rewards.append(time_step.reward.numpy()[0])
    
print("Total reward: ", sum(rewards))
print("Total steps: ", steps)


Map:
 [[[1 2 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 3]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
Direction:
 tf.Tensor([[ True  True  True False]], shape=(1, 4), dtype=bool)
Action:  [3]
Map:
 [[[2 0 0 0 0]
  [1 0 0 0 0]
  [0 0 0 0 3]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
Direction:
 tf.Tensor([[ True  True False False]], shape=(1, 4), dtype=bool)
Action:  [2]
Map:
 [[[0 0 0 0 0]
  [2 1 0 0 0]
  [0 0 0 0 3]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
Direction:
 tf.Tensor([[ True False False False]], shape=(1, 4), dtype=bool)
Action:  [2]
Map:
 [[[0 0 0 0 0]
  [0 2 1 0 0]
  [0 0 0 0 3]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
Direction:
 tf.Tensor([[ True False False False]], shape=(1, 4), dtype=bool)
Action:  [3]
Map:
 [[[0 0 0 0 0]
  [0 0 2 0 0]
  [0 0 1 0 3]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
Direction:
 tf.Tensor([[False  True False False]], shape=(1, 4), dtype=bool)
Action:  [3]
Map:
 [[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 2 0 3]
  [0 0 1 0 0]
  [0 0 0 0 0]]]
Direction:
 tf.Tensor([[False  True False False]], shape=(1, 4), dtype=bool)
Actio

In [ ]:
# Count the average steps per episode

n = 500
total_steps = 0
total_reward = 0
steps = 0
total_500 = 0

for i in range(n):
    if i % 10 == 0:
        print("Episode: ", i)
        
    time_step = env.reset()
    steps = 0
    while not time_step.is_last() and steps < 500:
        steps += 1
        action_step = agent.policy.action(time_step)
        time_step = env.step(action_step.action)
        reward = time_step.reward.numpy()[0]
        total_reward += reward
    
    if steps == 500:
        total_500 += 1
    else:
        total_steps += steps
        
if steps == 500:
    print("500 episodes reached")
    
print("Average steps per episode: ", total_steps / (n - total_500))
print("Average reward per episode: ", total_reward / n)
print("Total 500 episodes: ", total_500)


Episode:  0
Episode:  10
Episode:  20
Episode:  30
Episode:  40
Episode:  50
Episode:  60
Episode:  70
Episode:  80
Episode:  90
Average steps per episode:  85.19444444444444
Average reward per episode:  29.7
Total 500 episodes:  28


In [ ]:
# Save the q network weights
tf.keras.models.save_model(q_net._model, "q_network.h5")

/var/folders/bw/_7s8wxw93cngpt5f7bg5s3hm0000gn/T/ipykernel_13379/162919333.py:3: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(q_net._model, "q_network.h5")
